<a href="https://colab.research.google.com/github/akshatshaw/Stock-sentiment-analysis/blob/main/word_embbedding_sentiment_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
!pip install -q yfinance

In [ ]:
import yfinance as yf
import gensim

In [ ]:
df = pd.read_csv("/content/historical_nvidia_1.csv").drop(columns=['time','top_sentiment','sentiment_score','source'])

In [ ]:
df = df.groupby('date')['title'].agg(' '.join).reset_index()
df['Date']= pd.to_datetime(df['date'])
df.drop(columns=['date'], inplace = True)

In [ ]:
def process(data):
    data['P/L'] = data["Close"]-data['Open']
    data.drop(columns=['Volume','Adj Close'], inplace = True)
    return data

def pull_out(stock, data):
    price = pd.DataFrame(yf.download(stock, period="max"))
    # data = data[data.index>= '2023-12-31']
    price.reset_index(inplace = True)
    merged_df = pd.merge(data, price, on='Date', how='outer').dropna()
    return (merged_df)

In [ ]:
data = process(pull_out("NVDA", df))
#  = data['news_articles'].str.lower()
data['text'] = data.title.apply(gensim.utils.simple_preprocess)
data['target'] =data['P/L'].apply(lambda x: 1 if x > 0 else 0)
news = data.copy()
data.to_csv("NVDA_v0.1.csv", index= False)
news

[*********************100%%**********************]  1 of 1 completed


,title,Date,Open,High,Low,Close,P/L,text,target
0,Don’t Miss the Turnaround in These 3 Stocks,2022-10-04,12.879000,13.220000,12.879000,13.167000,0.288000,"[don, miss, the, turnaround, in, these, stocks]",1
1,YMTC Ban Likely A Plus For Micron & Western Di...,2022-10-05,12.910700,13.321000,12.621000,13.209000,0.298300,"[ymtc, ban, likely, plus, for, micron, western...",1
2,"With earnings on tap, tech titans like Microso...",2022-10-06,13.220000,13.656000,13.103000,13.130000,-0.090000,"[with, earnings, on, tap, tech, titans, like, ...",0
3,Should Investors Stay Away from Top Chipmaker ...,2022-10-07,12.505000,12.670000,12.022000,12.076000,-0.429000,"[should, investors, stay, away, from, top, chi...",0
5,"Why NVIDIA, Microsoft And Amazon Shares Traded...",2022-10-10,12.037000,12.124320,11.408120,11.670000,-0.367000,"[why, nvidia, microsoft, and, amazon, shares, ...",0
...,...,...,...,...,...,...,...,...,...
555,Immutep Collaborates With Merck To Evaluate Ef...,2024-06-03,113.621002,115.000000,112.002998,115.000000,1.378998,"[immutep, collaborates, with, merck, to, evalu...",1
556,Ivy League Inspiration: 3 Stocks That Harvard’...,2024-06-04,115.716003,116.599998,114.044998,116.436996,0.720993,"[ivy, league, inspiration, stocks, that, harva...",1
557,"Intel Stock (NASDAQ:INTC): It’s Game On, Even ...",2024-06-05,118.371002,122.449501,117.468002,122.440002,4.069000,"[intel, stock, nasdaq, intc, it, game, on, eve...",1
558,"Dear Apple Stock Fans, Mark Your Calendars for...",2024-06-06,124.047997,125.586998,118.320000,120.998001,-3.049995,"[dear, apple, stock, fans, mark, your, calenda...",0


In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# plt.plot(data['Open'])

In [ ]:
import gensim.downloader

glove_vectors = gensim.downloader.load('word2vec-google-news-300')


In [ ]:
def get_average_word2vec(tokens, model):
    valid_tokens = [token for token in tokens if token in model]
    if not valid_tokens:
        return np.zeros(model.vector_size)
    vectors = [model[token] for token in valid_tokens]
    return np.mean(vectors, axis=0)

# Apply the function to the 'tokens' column to get the embeddings
news['embedding'] = news['text'].apply(lambda x: get_average_word2vec(x, glove_vectors))

In [ ]:
# Convert embeddings to separate columns
embedding_df = pd.DataFrame(news['embedding'].to_list(), index=news.index)
embedding_df.columns = [f'embedding_{i}' for i in range(embedding_df.shape[1])]

# Concatenate the embeddings with the original DataFrame
news = pd.concat([news, embedding_df], axis=1).drop(columns=['embedding'])

In [ ]:
# news.to_csv('final_nvda_v1.0.csv',index = False)

news = pd.read_csv('final_nvda_v1.0.csv')

In [ ]:
news.head(1)

,title,Date,Open,High,Low,Close,P/L,text,target,embedding_0,...,embedding_290,embedding_291,embedding_292,embedding_293,embedding_294,embedding_295,embedding_296,embedding_297,embedding_298,embedding_299
0,Don’t Miss the Turnaround in These 3 Stocks,2022-10-04,12.879,13.22,12.879,13.167,0.288,"['don', 'miss', 'the', 'turnaround', 'in', 'th...",1,0.008275,...,0.040109,-0.047701,-0.093689,-0.003278,-0.007533,-0.016087,0.000087,-0.01491,0.129647,0.014404


In [ ]:
X = news.drop(columns=['Date','Close','title','P/L','target', 'text','High','Low'])
y=news['Close']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range = (1,4),max_features= 10000)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y,test_size=0.2,random_state=2)

In [ ]:
# total = pd.DataFrame(tfidf.fit_transform(data['text']).toarray())
# train = pd.DataFrame(tfidf.fit_transform(X_train['text']).toarray())
# test= pd.DataFrame(tfidf.fit_transform(X_test['text']).toarray())

In [ ]:
# !pip install catboost

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,f1_score,recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier

In [ ]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
bnb = BernoulliNB()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression(solver='liblinear', penalty='l2')
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
bc = BaggingClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50,random_state=2)
xgb = XGBClassifier(n_estimators=50,random_state=2)
# cat = CatBoostClassifier( silent = True)

In [ ]:
clfs = {
#     'SVC' : svc,
    'KN' : knc,
    # 'NB': mnb,
    'BNB': bnb,
    # 'cat':cat,
#     'DT': dtc,
    # 'LR': lrc,
    # 'RF': rfc,
#     'AdaBoost': abc,
#     'BgC': bc,
#     'ETC': etc,
    # 'GBDT':gbdt,
    'xgb':xgb
 }
from sklearn.model_selection import cross_val_score
def train_classifier(clf,X_train,y_train,X_test,y_test,total , y):
    csr_val = cross_val_score(clf, total, y, cv=3,scoring= "f1")
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    recall = recall_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    f1score = f1_score(y_test,y_pred)
    accuracy = accuracy_score(y_test,y_pred)
    return recall,precision ,f1score,accuracy, csr_val.mean()

for name,clf in clfs.items():

    recall,precision,f1score,accuracy,csr_val = train_classifier(clf, X_train,y_train,X_test,y_test,X , y)

    print("For ",name)
    print("recall - ",recall)
    print("Precision - ",precision)
    print("f1_score - ",f1score)
    print("accuracy - ",accuracy)
    print("cross_val_f1- ",csr_val)
    print("************************")

For  KN
recall -  0.7317073170731707
Precision -  0.5882352941176471
f1_score -  0.6521739130434783
accuracy -  0.6144578313253012
cross_val_f1-  0.5505116359397263
************************
For  BNB
recall -  0.7560975609756098
Precision -  0.543859649122807
f1_score -  0.6326530612244897
accuracy -  0.5662650602409639
cross_val_f1-  0.6026249880697321
************************
For  xgb
recall -  0.7804878048780488
Precision -  0.6153846153846154
f1_score -  0.6881720430107527
accuracy -  0.6506024096385542
cross_val_f1-  0.6039002267573697
************************


In [ ]:
X.columns =[i for i in range(303)]

##Try LSTM and BERT/ Transformer

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# Define custom metrics
def recall_m(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

# Build the neural network model
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(x_train.shape[1],)),\
    layers.Dropout( 0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout( 0.2),
    layers.Dense(32, activation='relu'),
    layers.Dropout( 0.2),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Use 'sigmoid' for binary classification
])

# Compile the model with custom metrics
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', precision_m, recall_m, f1_m])

# Train the model
history = model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc, test_precision, test_recall, test_f1 = model.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')
print(f'Test precision: {test_precision}')
print(f'Test recall: {test_recall}')
print(f'Test F1 score: {test_f1}')


Epoch 1/50
9/9 [==============================] - 2s 51ms/step - loss: 1.4250 - accuracy: 0.4735 - precision_m: 0.5128 - recall_m: 0.4606 - f1_m: 0.4526 - val_loss: 0.8515 - val_accuracy: 0.6061 - val_precision_m: 0.7292 - val_recall_m: 1.0000 - val_f1_m: 0.8249
Epoch 2/50
9/9 [==============================] - 0s 8ms/step - loss: 0.9585 - accuracy: 0.4659 - precision_m: 0.4983 - recall_m: 0.6381 - f1_m: 0.5574 - val_loss: 0.6683 - val_accuracy: 0.6061 - val_precision_m: 0.7292 - val_recall_m: 1.0000 - val_f1_m: 0.8249
Epoch 3/50
9/9 [==============================] - 0s 10ms/step - loss: 0.8502 - accuracy: 0.5114 - precision_m: 0.5401 - recall_m: 0.6346 - f1_m: 0.5712 - val_loss: 0.7114 - val_accuracy: 0.3939 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 4/50
9/9 [==============================] - 0s 9ms/step - loss: 0.7447 - accuracy: 0.5227 - precision_m: 0.5518 - recall_m: 0.7072 - f1_m: 0.6167 - val_loss: 0.7244 - val_accuracy: 0.3939 - val_